<a href="https://colab.research.google.com/github/haricharanvihari/extensive_viz/blob/master/S12_DNN/S12_EVA4S12F12_Haricharan_Vihari_N_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/S12_DNN/tiny-imagenet-200.zip", 'r')
zip_ref.extractall("/tiny-imagenet-200/")
zip_ref.close()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


## Model

In [6]:
!pip install albumentations==0.4.5

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

!pip install torchsummary
from torchsummary import summary

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

#Load Resnet18
model = models.resnet18()

model.avgpool = nn.AdaptiveAvgPool2d(1)
model.fc.out_features = 200
model = model.to(device)

summary(model, input_size=(3, 32, 32))

cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
       BasicBlock-11             [-1, 64, 8, 8]               0
           Conv2d-12             [-1, 64, 8, 8]          36,864
      BatchNorm2d-13             [-1, 64, 8, 8]             128
             ReLU-14             [-

## Run the Model

In [1]:
%matplotlib inline

In [4]:
import torch.nn as nn

from dataloader import ImageData
from datamodel import DataModel
from optimizer import OptimizerFactory

from torch.optim.lr_scheduler import OneCycleLR

dataset_path = '/content/drive/My Drive/Colab Notebooks/S12_DNN/tiny-imagenet-200/'
# Loading data and transforming it
img_data = ImageData()

# Choose from "albumentations" or "pytorch". Default is "pytorch"
img_data.load_TINY_IMAGENET(dataset_path, "albumentations")

criterion = nn.CrossEntropyLoss()

# optimizer can be choose from "SGD" & "Adam"
optim_sgd = OptimizerFactory("SGD")
optimizer = optim_sgd.load(params=model.parameters(), lr=0.02, momentum=0.9, weight_decay=1e-4)

dm = DataModel(img_data, criterion, optimizer, None, num_of_epochs = 30, cal_misclassified = True)

CUDA Available? False


KeyboardInterrupt: ignored

### Find LR Max

In [ ]:
lr = dm.find_lr_max(model, device)

### LR Max

In [ ]:
lr = 0.015
print(lr)

## Run Model

In [ ]:
# training the dataset and then running test
scheduler = OneCycleLR(optimizer, max_lr=lr, epochs=2, steps_per_epoch = len(img_data.trainloader), 
                          anneal_strategy='cos',
                          base_momentum=0.85,
                          cycle_momentum=True,
                          div_factor=10,
                          final_div_factor=10000.0,
                          last_epoch=-1,
                          max_momentum=0.95,
                          pct_start=0.3
                        )
dm.schedular = scheduler
dm.optimizer = optim_sgd.load(params=model.parameters(), lr=lr/10, momentum=0.9, weight_decay=0.005)
dm.run_model(model, device)

## Plotting Test & Train Accuracy

In [ ]:
# dm.plot_test_train_accuracy()

## Plotting Validation Accuracy

In [ ]:
# dm.plot_loss_accuracy()

## Plotting Misclassified Images

In [ ]:
# dm.plot_misclassified()

In [ ]:
# dm.plot_GRADcam(["layer1","layer2","layer3"])